Setup

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from scipy.optimize import minimize
from scipy.fft import fft, ifft
import pickle
from concurrent.futures import ProcessPoolExecutor

from kpz import *
from inverse import *

In [ ]:
x_bounds = np.array([100, 250])
t_bounds = np.array([10, 80])
# t_bounds = np.array([10, 13])

Load Daryn's data

In [ ]:
# load real data
filename = 'spread.pickle'
with open(filename, 'rb') as f:
    spread = pickle.load(f)

Run many simulations

In [ ]:
allErr = np.zeros(x_bounds[1]-x_bounds[0])
args = []
for t1 in range(t_bounds[0], t_bounds[1]):
    for t2 in range(t1+1, t_bounds[1]):
        for t3 in range(t1+1, t_bounds[1]):
            # catch the case when any of the times are the same
            if t1 == t2 or t2 == t3 or t1 == t3:
                continue
            args.append((t1, t2, t3))

a_opts = np.zeros((len(args), 3))

with ProcessPoolExecutor() as executor:
    futures = [executor.submit(runExperiment, spread, t1, t2, t3, x_bounds) for t1, t2, t3 in args]
    results = [f.result() for f in futures]

for i, r in enumerate(results):
    firsth, lasth, verifyh, ha_result, y_bounds, final, err = r
    allErr += np.abs(err)
    a_opts[i] = ha_result.x
            
plt.clf()
_ = plt.hist(allErr)
plt.show()

margin of error for c v lamb

find distributions for c v lamb

and find power law (in paper about burning paper)

basically, investigating what's wrong with the KPZ equation and what it's failing to capture. if c v lamb don't capture much if any useful information about the spread, then the model isn't so good and i'll need a new equation

markov chain mc model for c v lamb